In [1]:
!nvidia-smi

Sun Sep  5 19:11:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    61W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Download Data

In [ ]:
# download data
!wget http://bnci-horizon-2020.eu/database/data-sets/001-2014/A0{1..9}T.mat

In [ ]:
!wget http://bnci-horizon-2020.eu/database/data-sets/001-2014/A0{1..9}E.mat

# Imports

## Import Tensorflow 1.x version (actual 1.15) to use pretrained models provided by the code authors in https://github.com/okbalefthanded/eeg-tcnet/blob/master/Accuracy_and_kappa_scores.ipynb

In [ ]:
%tensorflow_version 1.x

In [2]:
%tensorflow_version 2.x

In [3]:
import tensorflow as tf
import numpy as np
import random
# 
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Define model

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv1D,Conv2D, AveragePooling2D,SeparableConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout, Add, Lambda,DepthwiseConv2D,Input, Permute
from tensorflow.keras.constraints import max_norm


def EEGTCNet(nb_classes,Chans=64, Samples=128, layers=3, kernel_s=10,filt=10, dropout=0, activation='relu', F1=4, D=2, kernLength=64, dropout_eeg=0.1):
    input1 = Input(shape = (1,Chans, Samples))
    input2 = Permute((3,2,1))(input1)
    regRate=.25
    numFilters = F1
    F2= numFilters*D

    EEGNet_sep = EEGNet(input_layer=input2,F1=F1,kernLength=kernLength,D=D,Chans=Chans,dropout=dropout_eeg)
    block2 = Lambda(lambda x: x[:,:,-1,:])(EEGNet_sep)
    outs = TCN_block(input_layer=block2,input_dimension=F2,depth=layers,kernel_size=kernel_s,filters=filt,dropout=dropout,activation=activation)
    out = Lambda(lambda x: x[:,-1,:])(outs)
    dense        = Dense(nb_classes, name = 'dense',kernel_constraint = max_norm(regRate))(out)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1,outputs=softmax, name="EEG-TCNET")

def EEGNet(input_layer=None,F1=4, kernLength=64, D=2, Chans=22, Samples=128, dropout=0.1, fullmodel=False, nb_classes=4):
    F2= F1*D
    norm_rate =.25
    if fullmodel:
        input1 = Input(shape = (1, Chans, Samples))
        input2 = Permute((3,2,1))(input1)
        input_layer = input2
    
    block1 = Conv2D(F1, (kernLength, 1), padding = 'same',data_format='channels_last',use_bias = False)(input_layer)
    block1 = BatchNormalization(axis = -1)(block1)
    block2 = DepthwiseConv2D((1, Chans), use_bias = False, 
                                    depth_multiplier = D,
                                    data_format='channels_last',
                                    depthwise_constraint = max_norm(1.))(block1)
    block2 = BatchNormalization(axis = -1)(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((8,1),data_format='channels_last')(block2)
    block2 = Dropout(dropout)(block2)
    block3 = SeparableConv2D(F2, (16, 1),
                            data_format='channels_last',
                            use_bias = False, padding = 'same')(block2)
    block3 = BatchNormalization(axis = -1)(block3)
    block3 = Activation('elu')(block3)
    block3 = AveragePooling2D((8,1),data_format='channels_last')(block3)
    block3 = Dropout(dropout)(block3)    

    if fullmodel:
        flatten = Flatten(name = 'flatten')(block3)    
        dense  = Dense(nb_classes, name = 'dense', kernel_constraint = max_norm(norm_rate))(flatten)
        softmax = Activation('softmax', name = 'softmax')(dense)
        return Model(inputs=input1, outputs=softmax, name="EEGNet")
    else:
        return block3

def TCN_block(input_layer,input_dimension,depth,kernel_size,filters,dropout,activation='relu'):
    block = Conv1D(filters,kernel_size=kernel_size,dilation_rate=1,activation='linear',
                   padding = 'causal',kernel_initializer='he_uniform')(input_layer)
    block = BatchNormalization()(block)
    block = Activation(activation)(block)
    block = Dropout(dropout)(block)
    block = Conv1D(filters,kernel_size=kernel_size,dilation_rate=1,activation='linear',
                   padding = 'causal',kernel_initializer='he_uniform')(block)
    block = BatchNormalization()(block)
    block = Activation(activation)(block)
    block = Dropout(dropout)(block)
    if(input_dimension != filters):
        conv = Conv1D(filters,kernel_size=1,padding='same')(input_layer)
        added = Add()([block,conv])
    else:
        added = Add()([block,input_layer])
    out = Activation(activation)(added)
    
    for i in range(depth-1):
        block = Conv1D(filters,kernel_size=kernel_size,dilation_rate=2**(i+1),activation='linear',
                   padding = 'causal',kernel_initializer='he_uniform')(out)
        block = BatchNormalization()(block)
        block = Activation(activation)(block)
        block = Dropout(dropout)(block)
        block = Conv1D(filters,kernel_size=kernel_size,dilation_rate=2**(i+1),activation='linear',
                   padding = 'causal',kernel_initializer='he_uniform')(block)
        block = BatchNormalization()(block)
        block = Activation(activation)(block)
        block = Dropout(dropout)(block)
        added = Add()([block, out])
        out = Activation(activation)(added)
        
    return out

# Preprocessing utils

In [5]:
import scipy.io as sio
import glob as glob
from tensorflow.keras.utils import to_categorical


def load_all_data (crossValidation, data_path): 

    big_X_train, big_y_train, big_X_test, big_y_test = [None]*9, [None]*9, [None]*9, [None]*9
    for subject in range (0,9):
        # path = data_path+'s' + str(subject+1) + '/'
        path = data_path
        big_X_train[subject], big_y_train[subject] = get_data(subject+1, True ,path)
        big_X_test[subject], big_y_test[subject] = get_data(subject+1, False ,path)
    
    return big_X_train, big_y_train, big_X_test, big_y_test

def get_data(subject,training,path, highpass = False):
	'''	Loads the dataset 2a of the BCI Competition IV
	available on http://bnci-horizon-2020.eu/database/data-sets
	Keyword arguments:
	subject -- number of subject in [1, .. ,9]
	training -- if True, load training data
				if False, load testing data
	
	Return:	data_return 	numpy matrix 	size = NO_valid_trial x 22 x 1750
			class_return 	numpy matrix 	size = NO_valid_trial
	'''
	NO_channels = 22
	NO_tests = 6*48 	
	Window_Length = 7*250 

	class_return = np.zeros(NO_tests)
	data_return = np.zeros((NO_tests,NO_channels,Window_Length))

	NO_valid_trial = 0
	if training:
		a = sio.loadmat(path+'A0'+str(subject)+'T.mat')
	else:
		a = sio.loadmat(path+'A0'+str(subject)+'E.mat')
	a_data = a['data']
	for ii in range(0,a_data.size):
		a_data1 = a_data[0,ii]
		a_data2= [a_data1[0,0]]
		a_data3= a_data2[0]
		a_X 		= a_data3[0]
		a_trial 	= a_data3[1]
		a_y 		= a_data3[2]
		a_fs 		= a_data3[3]
		a_classes 	= a_data3[4]
		a_artifacts = a_data3[5]
		a_gender 	= a_data3[6]
		a_age 		= a_data3[7]

		for trial in range(0,a_trial.size):
			if(a_artifacts[trial]==0):
				data_return[NO_valid_trial,:,:] = np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial])+Window_Length),:22])
				class_return[NO_valid_trial] = int(a_y[trial])
				NO_valid_trial +=1

	return data_return[0:NO_valid_trial,:,:], class_return[0:NO_valid_trial]

def prepare_features(path, subject, crossValidation=False):
    fs = 250 
    t1 = int(1.5*fs)
    t2 = int(6*fs)
    T = t2-t1
    X_train, y_train = get_data(subject+1,True,path)
    if crossValidation:
        X_train, X_test, y_train, y_test = train_test_split(
            X_train, y_train, test_size=0.2, random_state=0)
    else:
        X_test, y_test = get_data(subject+1,False , path)

    # prepare training data 	
    N_tr,N_ch,_ =X_train.shape 
    X_train = X_train[:,:,t1:t2].reshape(N_tr,1,N_ch,T)
    y_train_onehot = (y_train-1).astype(int)
    y_train_onehot = to_categorical(y_train_onehot)
    # prepare testing data 
    N_test, N_ch,_ =X_test.shape 
    X_test = X_test[:,:,t1:t2].reshape(N_test,1,N_ch,T)
    y_test_onehot = (y_test-1).astype(int)
    y_test_onehot = to_categorical(y_test_onehot)	

    return X_train,y_train,y_train_onehot,X_test,y_test,y_test_onehot

# Evaluate dataset

In [6]:
# from utils.models import EEGTCNet
# from utils.data_loading import prepare_features
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

data_path = '/content/'

F1 = 8
KE = 32
KT = 4
L = 2
FT = 12
pe = 0.2
pt = 0.3
classes = 4

channels = 22
crossValidation = False
batch_size = 64
epochs = 750
lr = 0.001

def evaluate_model(model):
    accs = []
    ks = []
    for subject in range(9):
        # path = data_path+'s{:}/'.format(subject+1)
        print(f"Evaluating subject: {subject+1}")
        path = data_path
        X_train, y_train, y_train_onehot, X_test, y_test, y_test_onehot = prepare_features(path, subject, crossValidation)   
    
        opt = Adam(lr=lr)
        model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['accuracy'])

        for j in range(22):
            scaler = StandardScaler()
            scaler.fit(X_train[:,0,j,:])
            X_train[:,0,j,:] = scaler.transform(X_train[:,0,j,:])
            X_test[:,0,j,:] = scaler.transform(X_test[:,0,j,:])

        model.fit(X_train, y_train_onehot, batch_size=batch_size, epochs=750, verbose=0)
        y_pred = model.predict(X_test).argmax(axis=-1)
        labels = y_test_onehot.argmax(axis=-1)
        accuracy_of_test = accuracy_score(labels, y_pred)
        kappa = cohen_kappa_score(labels, y_pred)
        accs.append(accuracy_of_test)
        ks.append(kappa)
        print(f"Model : {model.name} Subject: {subject+1} Acc: {accuracy_of_test} Kappa: {kappa}")

    print(f"Model : {model.name} Mean accuracy for all dataset: {np.mean(accs)} std. {np.std(accs)}")
    print(f"Model : {model.name} Mean Kappa for all dataset: {np.mean(ks)} std. {np.std(ks)}")

In [7]:
model = EEGTCNet(nb_classes = 4,Chans=22, Samples=1125, layers=L, kernel_s=KT,
                     filt=FT, dropout=pt, activation='elu', F1=F1, D=2, 
                     kernLength=KE, dropout_eeg=pe)
evaluate_model(model)

Evaluating subject: 1


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model : EEG-TCNET Subject: 1 Acc: 0.8042704626334519 Kappa: 0.7389931265093814
Evaluating subject: 2


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model : EEG-TCNET Subject: 2 Acc: 0.5441696113074205 Kappa: 0.39221855958445706
Evaluating subject: 3


...

Model : EEG-TCNET Subject: 7 Acc: 0.8700361010830325 Kappa: 0.8268629765955969
Evaluating subject: 8


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model : EEG-TCNET Subject: 8 Acc: 0.7822878228782287 Kappa: 0.7094493912411413
Evaluating subject: 9


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model : EEG-TCNET Subject: 9 Acc: 0.8295454545454546 Kappa: 0.7725706409372847
Model : EEG-TCNET Mean accuracy for all dataset: 0.7420152321254369 std. 0.12150931447444091
Model : EEG-TCNET Mean Kappa for all dataset: 0.6559286865066689 std. 0.161973484809704


In [8]:
model = EEGNet(F1=F1, kernLength=KE, D=2, Chans=22, Samples=1125, dropout=pt, fullmodel=True)
evaluate_model(model)   

Evaluating subject: 1


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model : EEGNet Subject: 1 Acc: 0.8220640569395018 Kappa: 0.7628331729714218
Evaluating subject: 2


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model : EEGNet Subject: 2 Acc: 0.568904593639576 Kappa: 0.4260207474398191
Evaluating subject: 3


...

Model : EEGNet Subject: 7 Acc: 0.776173285198556 Kappa: 0.7025374556161774
Evaluating subject: 8


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model : EEGNet Subject: 8 Acc: 0.7859778597785978 Kappa: 0.7145814418013438
Evaluating subject: 9


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model : EEGNet Subject: 9 Acc: 0.7803030303030303 Kappa: 0.7069810165339865
Model : EEGNet Mean accuracy for all dataset: 0.7337518389917769 std. 0.09498176115880937
Model : EEGNet Mean Kappa for all dataset: 0.6454441971259355 std. 0.12639322232753367


# Pretrained models

In [26]:
# dowload code
!git clone https://github.com/iis-eth-zurich/eeg-tcnet.git

Cloning into 'eeg-tcnet'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 61 (delta 16), reused 61 (delta 16), pack-reused 0
Unpacking objects: 100% (61/61), done.


## Accuracy and Kappa score calculation for EEG-TCNet

In [5]:
from joblib import dump, load
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline 
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

def build_model(path):
    model = load_model(path)
    #model = load_model(path +'best.h5')
    for l in model.layers:
        l.trainable = False
    lr = 0.001
    model.compile(loss = 'categorical_crossentropy',optimizer=Adam(lr=lr),metrics=['accuracy'])
    return model

class Scaler( BaseEstimator, TransformerMixin ):
    #Class Constructor 
    def __init__( self ):
        self.scalers = {}
        for j in range(22):
            self.scalers[j] = StandardScaler()
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        for j in range(22):
            self.scalers[j].fit(X[:,0 ,j, :])
        return self 
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        for j in range(22):
            X[:,0,j,:] = self.scalers[j].transform(X[:,0 ,j, :])
        return X

In [8]:
for i in range(9):
    clf = load('eeg-tcnet/models/EEG-TCNet/S{:}/pipeline_fixed.h5'.format(i+1)) 
    data_path = '/content/'
    path = data_path
    X_train,_,y_train_onehot,X_test,_,y_test_onehot = prepare_features(path,i,False)
    y_pred = clf.predict(X_test)
    acc_score = accuracy_score(y_pred,np.argmax(y_test_onehot,axis=1))
    kappa_score = cohen_kappa_score(y_pred,np.argmax(y_test_onehot,axis=1))
    print('For Subject: {:}, Accuracy: {:}, Kappa: {:}.'.format(i+1,acc_score*100, kappa_score))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


For Subject: 1, Accuracy: 85.76512455516014, Kappa: 0.8101928467695633.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


For Subject: 2, Accuracy: 65.01766784452296, Kappa: 0.5339432753888381.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


...

For Subject: 7, Accuracy: 87.36462093862815, Kappa: 0.8317628889235948.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


For Subject: 8, Accuracy: 83.76383763837639, Kappa: 0.7835188177411448.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


For Subject: 9, Accuracy: 78.03030303030303, Kappa: 0.7066441872940455.


## Accuracy and Kappa score calculation for Variable EEG-TCNet

In [10]:
for i in range(9):
    clf = load('eeg-tcnet/models/EEG-TCNet/S{:}/pipeline.h5'.format(i+1)) 
    data_path = '/content/'
    path = data_path
    X_train,_,y_train_onehot,X_test,_,y_test_onehot = prepare_features(path,i,False)
    y_pred = clf.predict(X_test)
    acc_score = accuracy_score(y_pred,np.argmax(y_test_onehot,axis=1))
    kappa_score = cohen_kappa_score(y_pred,np.argmax(y_test_onehot,axis=1))
    print('For Subject: {:}, Accuracy: {:}, Kappa: {:}.'.format(i+1,acc_score*100, kappa_score))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


For Subject: 1, Accuracy: 89.32384341637011, Kappa: 0.8576302100925488.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


For Subject: 2, Accuracy: 72.43816254416961, Kappa: 0.6325715331990611.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


...

For Subject: 7, Accuracy: 93.14079422382672, Kappa: 0.9085903848825899.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


For Subject: 8, Accuracy: 86.71586715867159, Kappa: 0.822837219437786.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


For Subject: 9, Accuracy: 85.22727272727273, Kappa: 0.8029247377689304.
